In [1]:
from gerrychain import Graph, Partition
import geopandas as gpd
from locality_split_scores import *

## pieces of counties

We will use the TX VTDs shapefile that was used in the VRA paper, [here](https://www.dropbox.com/sh/k78n2hyixmv9xdg/AABmZG5ntMbXtX1VKThR7_t8a?dl=0).

In [3]:
%%time
g = Graph.from_file("TX_VTDs/TX_VTDs.shp") # this may take a couple minutes

/Users/gabe/Desktop/mggg/GerryChain/gerrychain/graph/adjacency.py:66: UserWarning: Found overlaps among the given polygons. Indices of overlaps: {(1165, 1170), (7889, 8135), (603, 762), (638, 695), (6531, 6548), (6573, 6618), (7539, 7582), (763, 1812), (3500, 3510), (499, 735), (6519, 6537), (5970, 8790), (628, 629), (3871, 3886), (5974, 8790), (1159, 1168), (2981, 8790), (1806, 3877), (1114, 1154), (612, 733), (445, 446), (873, 982), (605, 639), (3496, 3505), (1663, 1686), (824, 860), (1163, 1165), (2930, 2938), (657, 658), (904, 978), (1478, 1479), (2801, 2818), (6551, 6580), (1478, 1488), (1577, 1683), (2875, 2904), (2811, 2813), (2805, 2827), (1661, 1728), (2876, 2878), (5414, 6142), (8780, 8792), (656, 714), (7457, 7526), (3505, 3510), (324, 387), (1761, 1769), (1636, 1671), (322, 344), (2984, 8790), (4608, 6563), (2817, 2828), (6539, 6608), (6591, 6609), (2778, 2818), (3544, 3550), (8743, 8748), (572, 5833), (595, 758), (362, 6405), (1113, 1175), (1637, 1663), (343, 348), (1113, 

CPU times: user 1min 55s, sys: 3.15 s, total: 1min 58s
Wall time: 1min 58s


In [3]:
g.nodes[0]

{'boundary_node': False,
 'area': 0.0005367553233252686,
 'CNTY_x': 1,
 'COLOR_x': 6,
 'VTD_x': '0001',
 'CNTYVTD': '10001',
 'VTDKEY_x': 1,
 'Shape_area': 5616315.63811,
 'Shape_len_': 15020.5442123,
 'Unnamed_ 0': 0,
 'TOTPOP_x': 3131.0,
 'NH_WHITE': 2053.0,
 'NH_BLACK': 560.0,
 'NH_AMIN': 12.0,
 'NH_ASIAN': 54.0,
 'NH_NHPI': 1.0,
 'NH_OTHER': 3.0,
 'NH_2MORE': 47.0,
 'HISP': 401.0,
 'H_WHITE': 193.0,
 'H_BLACK': 6.0,
 'H_AMIN': 14.0,
 'H_ASIAN': 0.0,
 'H_NHPI': 0.0,
 'H_OTHER': 160.0,
 'H_2MORE': 28.0,
 'VAP': 2341.0,
 'HVAP': 217.0,
 'WVAP': 1677.0,
 'BVAP': 368.0,
 'AMINVAP': 11.0,
 'ASIANVAP': 45.0,
 'NHPIVAP': 1.0,
 'OTHERVAP': 3.0,
 '2MOREVAP': 19.0,
 'white_prop': 0.716360529688167,
 'black_prop': 0.15719777872704,
 'hisp_prop': 0.092695429303716,
 'CD': '4',
 'Unnamed__1': 0,
 'RomneyR_12': 754,
 'ObamaD_12G': 261,
 'JohnsonL_1': 10,
 'SteinG_12G': 1,
 'Write-In _': 0,
 'CruzR_12G_': 713,
 'SadlerD_12': 285,
 'MyersL_12G': 13,
 'CollinsG_1': 8,
 'AllenD_12P': 0,
 'HubbardD_1'

In [4]:
county_splits = LocalitySplits(
            "countysplits",
            "CNTY_x",
            "TOTPOP_x",
            ["num_parts", "num_pieces","num_split_localities"]
        )

In [5]:
partition = Partition(g, "sldl358", updaters={"county_splits" : county_splits})

In [6]:
partition["county_splits"]

{'num_parts': 349, 'num_pieces': 361, 'num_split_localities': 24}

So the number of county pieces is 361, given the TX state House districting plan

## pieces of places
The places shapefile was found [here](https://catalog.data.gov/dataset/tiger-line-shapefile-2017-state-texas-current-place-state-based).

In [79]:
%%time
place_df = gpd.read_file("tl_2017_48_place/tl_2017_48_place.shp")
HD_df = gpd.read_file("Texas_State_House_Districts-shp/State_House_District.shp")

CPU times: user 490 ms, sys: 25.1 ms, total: 515 ms
Wall time: 519 ms


In [82]:
len(place_df), len(HD_df)

(1747, 150)

Loading the two above shapefiles into QGIS and taking the intersection of the two layers (Vector -> Geoprocessing Tools -> Intersection) gives us a layer with precisely 2354 units. This is the number of "place pieces" given the TX state House districting plan.